# Create Agent with User Confirmation

In this notebook we will create an Agent for Amazon Bedrock using the new capabilities for function definition with get user confirmation before invoking action group function.

We will use the HR agent as example. With this agent, you can request a new vacation leave. We will use a local function to define the logic that checks for the available vacation days and book new ones. Notice that this example does not need an API Schema file, nor does it need a Lambda function.

For this example, we will generate some employee data using an [SQLite](https://www.sqlite.org/) database

## Pre-requisites

Before starting, let's update the botocore and boto3 packages to ensure we have the latest version

In [113]:
!python3 -m pip install --upgrade -q botocore
!python3 -m pip install --upgrade -q boto3
!python3 -m pip install --upgrade -q awscli

Let's now check the boto3 version to ensure the correct version has been installed. Your version should be bigger or equal to 1.34.90.

In [13]:
import boto3
import json
import time
import uuid
import pprint
import logging

print(boto3.__version__)

1.35.23


In [14]:
# setting logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

Let's now create the boto3 clients for the required AWS services

In [15]:
# getting boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

Next we can set some configuration variables for the agent

In [16]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

('us-east-1', '336118322861')

In [17]:
# configuration variables
suffix = f"{region}-{account_id}"
agent_name = "hr-assistant-function-roc"
agent_bedrock_allow_policy_name = f"{agent_name}-ba-{suffix}"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'
agent_foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"
agent_description = "Agent for providing HR assistance to book holidays"
agent_instruction = "You are an HR agent, helping employees understand HR policies and manage vacation time"
agent_action_group_name = "VacationsActionGroup"
agent_action_group_description = "Actions to book new vacations in the system with user confirmation"
agent_alias_name = f"{agent_name}-alias"

## Create Agent

We will now create the agent. To do so, we first need to create the agent policies that allow bedrock model invocation and the agent IAM role with the policy associated to it. We will allow this agent to invoke the Claude Sonnet model

In [18]:
# Create IAM policies for agent
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{agent_foundation_model}"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)

In [19]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': '55d98c5b-48e4-46e2-82d7-51d61464e4be',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 20 Sep 2024 17:35:20 GMT',
   'x-amzn-requestid': '55d98c5b-48e4-46e2-82d7-51d61464e4be',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

## Creating agent

Once the needed IAM role is created, we can use the Bedrock agent client to create a new agent. To do so we use the create_agent function. It requires an agent name, underlying foundation model and instructions. You can also provide an agent description. Note that the agent created is not yet prepared. We will focus on preparing the agent and then using it to invoke actions and use other APIs.

In [20]:
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=agent_foundation_model,
    instruction=agent_instruction,
)
response

{'ResponseMetadata': {'RequestId': 'df2c33bc-2efb-4b16-8a0f-8d74f9339d16',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Fri, 20 Sep 2024 17:35:20 GMT',
   'content-type': 'application/json',
   'content-length': '637',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'df2c33bc-2efb-4b16-8a0f-8d74f9339d16',
   'x-amz-apigw-id': 'eajN7G4BoAMEemw=',
   'x-amzn-trace-id': 'Root=1-66edb258-260fb01067ba5b633df10f20'},
  'RetryAttempts': 0},
 'agent': {'agentArn': 'arn:aws:bedrock:us-east-1:336118322861:agent/M8LOXKHHNB',
  'agentId': 'M8LOXKHHNB',
  'agentName': 'hr-assistant-function-roc',
  'agentResourceRoleArn': 'arn:aws:iam::336118322861:role/AmazonBedrockExecutionRoleForAgents_hr-assistant-function-roc',
  'agentStatus': 'CREATING',
  'createdAt': datetime.datetime(2024, 9, 20, 17, 35, 20, 907735, tzinfo=tzlocal()),
  'description': 'Agent for providing HR assistance to book holidays',
  'foundationModel': 'anthropic.claude-3-sonnet-20240229-v1:0',
  'idleSessionTTLInSecond

Let's now store the agent id in a local variable to use it on the next steps

In [21]:
agent_id = response['agent']['agentId']
agent_id

'M8LOXKHHNB'

## Create Agent Action Group

We will now create an agent action group. The create_agent_action_group function provides this functionality. We will use DRAFT as the agent version since we haven't yet create an agent version or alias. To inform the agent about the action group functionalities, we will provide an action group description containing the functionalities of the action group.

In this example, we will provide the Action Group functionality using a functionSchema. You can also provide and APISchema. The notebook [02-create-agent-with-api-schema.ipynb](02-create-agent-with-api-schema/02-create-agent-with-api-schema.ipynb) provides an example of it.

To define the functions using a function schema, you need to provide the name, description, parameters and requireConfirmation for each function.

When defining functions using a function schema, you must specify the name, description, parameters, and the requireConfirmation attribute for each function. The requireConfirmation attribute indicates whether user confirmation is required before executing the function. 

For more details on this configuration, refer to the documentation at the provided [link](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-userconfirmation.html).

In [22]:
agent_functions = [

    {
        'name': 'reserve_vacation_time',
        'description': 'reserve vacation time for a specific employee - you need all parameters to reserve vacation time',
        'parameters': {
            "employee_id": {
                "description": "the id of the employee for which time off will be reserved",
                "required": True,
                "type": "integer"
            },
            "start_date": {
                "description": "the start date for the vacation time",
                "required": True,
                "type": "String"
            },
            "end_date": {
                "description": "the end date for the vacation time",
                "required": True,
                "type": "String"
            }
        },
        'requireConfirmation':'ENABLED'
    },
]

Here we create the action group with a `customContrl` executor of `RETURN_CONTROL`. This lets the agent know that instead of executing the function, it should simply return the appropriate function and parameters. The client application is then responsible for executing the function.

In [23]:
# Pause to make sure agent is created
time.sleep(30)
# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'customControl': 'RETURN_CONTROL'
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)

In [24]:
agent_action_group_response

{'ResponseMetadata': {'RequestId': '43f6a679-11dd-42ea-9159-3fe96b7d54db',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 20 Sep 2024 17:36:06 GMT',
   'content-type': 'application/json',
   'content-length': '943',
   'connection': 'keep-alive',
   'x-amzn-requestid': '43f6a679-11dd-42ea-9159-3fe96b7d54db',
   'x-amz-apigw-id': 'eajVDFFtIAMEXNQ=',
   'x-amzn-trace-id': 'Root=1-66edb286-5902af1c75c63571290c7d32'},
  'RetryAttempts': 0},
 'agentActionGroup': {'actionGroupExecutor': {'customControl': 'RETURN_CONTROL'},
  'actionGroupId': 'RQCZXNDCH5',
  'actionGroupName': 'VacationsActionGroup',
  'actionGroupState': 'ENABLED',
  'agentId': 'M8LOXKHHNB',
  'agentVersion': 'DRAFT',
  'createdAt': datetime.datetime(2024, 9, 20, 17, 36, 6, 561049, tzinfo=tzlocal()),
  'description': 'Actions to book new vacations in the system with user confirmation',
  'functionSchema': {'functions': [{'description': 'reserve vacation time for a specific employee - you need all parameters to rese

## Preparing Agent

Let's create a DRAFT version of the agent that can be used for internal testing.

In [63]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)

{'ResponseMetadata': {'RequestId': '9f8757fe-66fb-4148-b5ef-5a7c2b692559', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Fri, 20 Sep 2024 17:51:38 GMT', 'content-type': 'application/json', 'content-length': '119', 'connection': 'keep-alive', 'x-amzn-requestid': '9f8757fe-66fb-4148-b5ef-5a7c2b692559', 'x-amz-apigw-id': 'ealmnHFcoAMEDQg=', 'x-amzn-trace-id': 'Root=1-66edb62a-6478ee5047c52ef753b4461d'}, 'RetryAttempts': 1}, 'agentId': 'M8LOXKHHNB', 'agentStatus': 'PREPARING', 'agentVersion': 'DRAFT', 'preparedAt': datetime.datetime(2024, 9, 20, 17, 51, 38, 136455, tzinfo=tzlocal())}


## Invoke Agent

Now that we've created the agent, let's use the `bedrock-agent-runtime` client to invoke this agent and perform some tasks.

In [87]:
# Pause to make sure agent is prepared
time.sleep(30)

# Extract the agentAliasId from the response
agent_alias_id = "TSTALIASID"

In [104]:
## create a random id for session initiator id
session_id:str = str(uuid.uuid1())
enable_trace:bool = False
end_session:bool = False
# Pause to make sure agent alias is ready
# time.sleep(30)

# invoke the agent API
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="Book 2 days off for employee 1 with the start date of 2024-11-14 and the end date of 2024-11-19",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/json',
                                      'date': 'Fri, 20 Sep 2024 18:07:21 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '2e9aab48-777b-11ef-9c68-0affee3e2ef7',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': 'efe1243f-dcd2-4f55-994a-4e9c54d4ea64'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'efe1243f-dcd2-4f55-994a-4e9c54d4ea64',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x7f06dce42290>,
 'contentType': 'application/json',
 'sessionId': '2e9aab48-777b-11ef-9c68-0affee3e2ef7'}


In [105]:
%%time
event_stream = agentResponse['completion']

for event in event_stream:
    if 'returnControl' in event:
        pprint.pp(event)

{'returnControl': {'invocationId': '29e12a37-f18c-4a3e-bb19-c5e77f043223-uc-result',
                   'invocationInputs': [{'functionInvocationInput': {'actionGroup': 'VacationsActionGroup',
                                                                     'actionInvocationType': 'USER_CONFIRMATION_AND_RESULT',
                                                                     'function': 'reserve_vacation_time',
                                                                     'parameters': [{'name': 'end_date',
                                                                                     'type': 'string',
                                                                                     'value': '2024-11-19'},
                                                                                    {'name': 'employee_id',
                                                                                     'type': 'integer',
                                               

## Defining function implementation

Let's now implement our functions to get the vacation information for an employee_id and to book a vacation for an employee_id between start_date and end_dates.

To do so, we will first create an SQLite database with generated data

In [106]:
# creating employee database to be used by lambda function
import sqlite3
import random
from datetime import date, timedelta

# Connect to the SQLite database (creates a new one if it doesn't exist)
conn = sqlite3.connect('employee_database.db')
c = conn.cursor()

# Create the employees table
c.execute('''CREATE TABLE IF NOT EXISTS employees
                (employee_id INTEGER PRIMARY KEY AUTOINCREMENT, employee_name TEXT, employee_job_title TEXT, employee_start_date TEXT, employee_employment_status TEXT)''')

# Create the vacations table
c.execute('''CREATE TABLE IF NOT EXISTS vacations
                (employee_id INTEGER, year INTEGER, employee_total_vacation_days INTEGER, employee_vacation_days_taken INTEGER, employee_vacation_days_available INTEGER, FOREIGN KEY(employee_id) REFERENCES employees(employee_id))''')

# Create the planned_vacations table
c.execute('''CREATE TABLE IF NOT EXISTS planned_vacations
                (employee_id INTEGER, vacation_start_date TEXT, vacation_end_date TEXT, vacation_days_taken INTEGER, FOREIGN KEY(employee_id) REFERENCES employees(employee_id))''')

# Generate some random data for 10 employees
employee_names = ['John Doe', 'Jane Smith', 'Bob Johnson', 'Alice Williams', 'Tom Brown', 'Emily Davis', 'Michael Wilson', 'Sarah Taylor', 'David Anderson', 'Jessica Thompson']
job_titles = ['Manager', 'Developer', 'Designer', 'Analyst', 'Accountant', 'Sales Representative']
employment_statuses = ['Active', 'Inactive']

for i in range(10):
    name = employee_names[i]
    job_title = random.choice(job_titles)
    start_date = date(2015 + random.randint(0, 7), random.randint(1, 12), random.randint(1, 28)).strftime('%Y-%m-%d')
    employment_status = random.choice(employment_statuses)
    c.execute("INSERT INTO employees (employee_name, employee_job_title, employee_start_date, employee_employment_status) VALUES (?, ?, ?, ?)", (name, job_title, start_date, employment_status))
    employee_id = c.lastrowid

    # Generate vacation data for the current employee
    for year in range(date.today().year, date.today().year - 3, -1):
        total_vacation_days = random.randint(10, 30)
        days_taken = random.randint(0, total_vacation_days)
        days_available = total_vacation_days - days_taken
        c.execute("INSERT INTO vacations (employee_id, year, employee_total_vacation_days, employee_vacation_days_taken, employee_vacation_days_available) VALUES (?, ?, ?, ?, ?)", (employee_id, year, total_vacation_days, days_taken, days_available))

        # Generate some planned vacations for the current employee and year
        num_planned_vacations = random.randint(0, 3)
        for _ in range(num_planned_vacations):
            start_date = date(year, random.randint(1, 12), random.randint(1, 28)).strftime('%Y-%m-%d')
            end_date = (date(int(start_date[:4]), int(start_date[5:7]), int(start_date[8:])) + timedelta(days=random.randint(1, 14))).strftime('%Y-%m-%d')
            days_taken = (date(int(end_date[:4]), int(end_date[5:7]), int(end_date[8:])) - date(int(start_date[:4]), int(start_date[5:7]), int(start_date[8:])))
            c.execute("INSERT INTO planned_vacations (employee_id, vacation_start_date, vacation_end_date, vacation_days_taken) VALUES (?, ?, ?, ?)", (employee_id, start_date, end_date, days_taken.days))

# Commit the changes and close the connection
conn.commit()
conn.close()

Next let's use the generated file `employee_database.db` to provide the results for our query by implementing `reserve_vacation_time`

In [107]:
import sqlite3
from datetime import datetime
    
def reserve_vacation_time(employee_id, start_date, end_date):
    # Connect to the SQLite database
    conn = sqlite3.connect('employee_database.db')
    c = conn.cursor()
    try:
        # Calculate the number of vacation days
        start_date = datetime.strptime(start_date, '%Y-%m-%d')
        end_date = datetime.strptime(end_date, '%Y-%m-%d')
        vacation_days = (end_date - start_date).days + 1

        # Get the current year
        current_year = start_date.year

        # Check if the employee exists
        c.execute("SELECT * FROM employees WHERE employee_id = ?", (employee_id,))
        employee = c.fetchone()
        if employee is None:
            return_msg = f"Employee with ID {employee_id} does not exist."
            print(return_msg)
            conn.close()
            return return_msg

        # Check if the vacation days are available for the employee in the current year
        query = """SELECT COALESCE(
                        (SELECT employee_vacation_days_available 
                         FROM vacations 
                         WHERE employee_id = ? 
                         AND year = ?), 0) AS employee_vacation_days_available;"""
        
        c.execute(query, (employee_id, current_year))
        result = c.fetchone()
        available_days = result[0] if result else 0
        
        
        print(available_days)
        
        if available_days is None or available_days < vacation_days:
            return_msg = f"Employee with ID {employee_id} does not have enough vacation days available for the requested period."
            print(return_msg)
            conn.close()
            return return_msg

        # Insert the new vacation into the planned_vacations table
        c.execute("INSERT INTO planned_vacations (employee_id, vacation_start_date, vacation_end_date, vacation_days_taken) VALUES (?, ?, ?, ?)", (employee_id, start_date, end_date, vacation_days))

        # Update the vacations table with the new vacation days taken
        c.execute("UPDATE vacations SET employee_vacation_days_taken = employee_vacation_days_taken + ?, employee_vacation_days_available = employee_vacation_days_available - ? WHERE employee_id = ? AND year = ?", (vacation_days, vacation_days, employee_id, current_year))

        conn.commit()
        print(f"Vacation booked successfully for employee with ID {employee_id} from {start_date} to {end_date}.")
        # Close the database connection
        conn.close()
        return f"Vacation booked successfully for employee with ID {employee_id} from {start_date} to {end_date}."
    except Exception as e:
        conn.rollback()
        # Close the database connection
        conn.close()
        raise Exception(f"Error occurred: {e}")

print(event["returnControl"]["invocationInputs"][0]["functionInvocationInput"]["function"])

reserve_vacation_time


We can now call the `reserve_vacation_time`function with the parameters provided by the agent

In [108]:
reserve_vacation_time_msg = None
for invocationInput in event["returnControl"]["invocationInputs"]:
    function_to_call = invocationInput["functionInvocationInput"]["function"]
    if function_to_call == "reserve_vacation_time":
        employee_id = None
        start_date = None
        end_date = None
        for param in invocationInput["functionInvocationInput"]["parameters"]:
            if param["name"] == "employee_id":
                employee_id = param["value"]
            if param["name"] == "start_date":
                start_date = param["value"]
            if param["name"] == "end_date":
                end_date = param["value"]
                
        if employee_id and start_date and end_date:
            reserve_vacation_time_msg = reserve_vacation_time(employee_id, start_date, end_date)

reserve_vacation_time_msg

0
Employee with ID 1 does not have enough vacation days available for the requested period.


'Employee with ID 1 does not have enough vacation days available for the requested period.'

## Invoking the agent with the function results

Finally, we need to invoke the agent passing the function results as a parameter. This lets us use the agent for generating the final response.

Note that if user confirmation is enabled for the function or API, the user will be presented with the option to `CONFIRM` or `DENY` the action specified in the response, within the `confirmationState` field.

If the user's response is `CONFIRM`, the function or API in the response will be executed.

If the user's response is `DENY`, the function or API in the response will not be executed.

In [109]:
raw_response_with_roc = bedrock_agent_runtime_client.invoke_agent(
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    sessionState={
        'invocationId': event["returnControl"]["invocationId"],
        'returnControlInvocationResults': [{
                'functionResult': {
                    'actionGroup': event["returnControl"]["invocationInputs"][0]["functionInvocationInput"]["actionGroup"],
                    'function': event["returnControl"]["invocationInputs"][0]["functionInvocationInput"]["function"],
                    'confirmationState': 'CONFIRM',
                    'responseBody': {
                        "TEXT": {
                            'body': "reserve_vacation_time: "+str(reserve_vacation_time_msg)
                        }
                    }
                }
        }]}
)

print(raw_response_with_roc)

{'ResponseMetadata': {'RequestId': 'ba795b81-7eb3-4fe5-9782-1285965cc159', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 20 Sep 2024 18:07:47 GMT', 'content-type': 'application/json', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'x-amzn-requestid': 'ba795b81-7eb3-4fe5-9782-1285965cc159', 'x-amz-bedrock-agent-session-id': '2e9aab48-777b-11ef-9c68-0affee3e2ef7', 'x-amzn-bedrock-agent-content-type': 'application/json'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'sessionId': '2e9aab48-777b-11ef-9c68-0affee3e2ef7', 'completion': <botocore.eventstream.EventStream object at 0x7f06dce441f0>}


In [112]:
%%time
event_stream = raw_response_with_roc['completion']
for event in event_stream:
    print(event)

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 9.54 µs


In [111]:
print(raw_response_with_roc)

{'ResponseMetadata': {'RequestId': 'ba795b81-7eb3-4fe5-9782-1285965cc159', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 20 Sep 2024 18:07:47 GMT', 'content-type': 'application/json', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'x-amzn-requestid': 'ba795b81-7eb3-4fe5-9782-1285965cc159', 'x-amz-bedrock-agent-session-id': '2e9aab48-777b-11ef-9c68-0affee3e2ef7', 'x-amzn-bedrock-agent-content-type': 'application/json'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'sessionId': '2e9aab48-777b-11ef-9c68-0affee3e2ef7', 'completion': <botocore.eventstream.EventStream object at 0x7f06dce441f0>}


## Clean up (optional)

The next steps are optional and demonstrate how to delete our agent. To delete the agent we need to:

1. update the action group to disable it
2. delete agent action group
3. delete agent
4. delete the created IAM roles and policies

In [114]:
# This is not needed, you can delete agent successfully after deleting alias only
# Additionaly, you need to disable it first
action_group_id = agent_action_group_response['agentActionGroup']['actionGroupId']
action_group_name = agent_action_group_response['agentActionGroup']['actionGroupName']

response = bedrock_agent_client.update_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id,
    actionGroupName=action_group_name,
    actionGroupExecutor={
        'customControl': 'RETURN_CONTROL'
    },
    functionSchema={
        'functions': agent_functions
    },
    actionGroupState='DISABLED',
)

action_group_deletion = bedrock_agent_client.delete_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id
)

In [115]:
agent_deletion = bedrock_agent_client.delete_agent(
    agentId=agent_id
)

In [116]:
# Delete IAM Roles and policies

for policy in [agent_bedrock_allow_policy_name]:
    iam_client.detach_role_policy(RoleName=agent_role_name, PolicyArn=f'arn:aws:iam::{account_id}:policy/{policy}')
    
for role_name in [agent_role_name]:
    iam_client.delete_role(
        RoleName=role_name
    )

for policy in [agent_bedrock_policy]:
    iam_client.delete_policy(
        PolicyArn=policy['Policy']['Arn']
)

## Thank you!